In [107]:
import pandas as pd
from urllib.request import urlopen
import certifi
import ssl
import json
import fmpsdk
from dateutil.relativedelta import relativedelta
from openpyxl.utils import get_column_letter



# Create a secure SSL context
context_init = ssl.create_default_context(cafile=certifi.where())


In [108]:
def get_data_url(url):
    response = urlopen(url, context=context_init)
    data = response.read().decode("utf-8")
    data=json.loads(data)
    return pd.DataFrame(data)

def get_org_chart(ticker):
    apikey='2723105eaade3c1ca8d66ca5c567b590'
    url = (f'https://financialmodelingprep.com/api/v3/key-executives/{ticker}?apikey={apikey}')
    df_=get_data_url(url)
    df_=df_[['title']+['name']]
    df_=df_.drop_duplicates(subset='name')
    #df_=df_.drop_duplicates(subset='title', inplace=True)
    return df_

def stock_price(ticker):
    try:
        stock_prices = fmpsdk.historical_price_full(apikey='2723105eaade3c1ca8d66ca5c567b590', symbol=ticker)
        closing_prices = [price['close'] for price in stock_prices]

        weekly_stock_variation = (closing_prices[0]-closing_prices[7])/closing_prices[7]

        if weekly_stock_variation >=0:
            return "+"+str(round(weekly_stock_variation*100,2))+"%"
        else:
            return str(round(weekly_stock_variation*100,2))+"%"
        
    except:
        return "FAIL"
    

def data_inventory(ticker,period):
    
    apikey='2723105eaade3c1ca8d66ca5c567b590'
    
    symbol: str = ticker
    
    try:
        data_BSS=pd.DataFrame(fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol, period=period))
        #return '$'+str((data_BSS['inventory'].iloc[0])/1000000)+"M"
        return data_BSS['inventory'].iloc[0]
    except:
        return "FAIL"


def get_latest_earnings_transcript(ticker):
    apikey = '2723105eaade3c1ca8d66ca5c567b590'
    
    url = f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?apikey={apikey}'
    
    try:
        data = get_data_url(url)
        date_latest=pd.to_datetime(data["date"].iloc[0])
        date_next=date_latest + relativedelta(months=3)
        transcript=data['content'].iloc[0]
        return transcript, date_next
    except:
        return 'FAIL', 'FAIL'

def return_company_profile(tck):
    # Get company profile and industry from the ticker
    company_info = fmpsdk.company_profile(symbol=tck, apikey='2723105eaade3c1ca8d66ca5c567b590')
    try:
        return pd.DataFrame([company_info[0]])
    except: 
        return pd.DataFrame([{'companyName': 'FAIL', 'industry': 'FAIL'}])

def new_executive(tck):
    return 'in dev.'

def credit_rating(tck):
    df_rating=pd.read_csv('TCC companies mapping 10142024.csv')
    df_rating_filtered=df_rating[df_rating['ticker']==tck]
    
    if df_rating_filtered.empty:
        return 'Need Manual research'
    else:
        return df_rating_filtered['S&P rating'].iloc[0]

In [109]:
def generate_table(lst_tck):
    df_rtr = pd.DataFrame(columns=['Company Name', 'Ticker', 'Industry', 'Stock Trend (7days)', 'Inventory ($)', 'S&P Rating (2023)','Next Earnings', 'Latest Earnings Presentation','Exec. Update'])
    for tck in lst_tck:
        print(tck)
        company_profil=return_company_profile(tck)
        data_earning_=get_latest_earnings_transcript(tck)
        
        stock_trend=stock_price(tck)
        next_call=data_earning_[1]
        transcript=data_earning_[0]
        inventory=data_inventory(tck, 'annual')
        company_name=company_profil['companyName'].iloc[0]
        company_industry=company_profil['industry'].iloc[0]
        credit_SnP=credit_rating(tck)
        exec_update=new_executive(tck)
        
        df_rtr.loc[len(df_rtr)]=[company_name, tck, company_industry, stock_trend, inventory, credit_SnP, next_call, transcript, exec_update]
            
    #df_rtr.to_excel('Target List Intelligence.xlsx', index=False)
    return df_rtr


def generate_output(list_tck):
    df=generate_table(list_tck)
    with pd.ExcelWriter('Target List Intelligence.xlsx', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Target List')
        worksheet = writer.sheets['Target List']

        for i in range(len(df.columns)):
            col_letter = get_column_letter(i + 1)  # 1-based indexing
            worksheet.column_dimensions[col_letter].width = 25  # or any width you want

In [110]:
return_company_profile('BRK.B')

,companyName,industry
0,FAIL,FAIL


In [111]:
tickers = ['MSFT', 'TSLA', 'AAPL', 'GOOG', 'AMZN', 'META', 'NVDA', 'BRK.B', 'JPM', 'UNH']

generate_output(tickers)

MSFT


C:\Users\thoir\AppData\Local\Temp\ipykernel_21600\3775500943.py:46: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


TSLA


C:\Users\thoir\AppData\Local\Temp\ipykernel_21600\3775500943.py:46: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


AAPL


...

C:\Users\thoir\AppData\Local\Temp\ipykernel_21600\3775500943.py:46: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


JPM


C:\Users\thoir\AppData\Local\Temp\ipykernel_21600\3775500943.py:46: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


UNH


C:\Users\thoir\AppData\Local\Temp\ipykernel_21600\3775500943.py:46: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


# Git COMMIT

In [75]:
!git add .


In [76]:
!git commit -m "new columns to the DF"


[main 1930280] new columns to the DF
 2 files changed, 212 insertions(+), 48 deletions(-)


In [77]:
!git push origin main


To https://github.com/alexandre-tcc/target-client_tracking.git
   9ec9506..1930280  main -> main
